# 영상 시각적 특징 분석

- 1배속 영상을 넣었을 때, 모델이 1, 1.1, 1.2, 1.3 배속 중 어떤 것 같은지 판단함

- Ex) 1배속 영상을 넣었는데, 1.3배속이라고 판단함 -> 이 영상의 특징은 무엇인가? (TI? SI? ...)

In [2]:
import utils
import os
import shutil
import cv2

# 비디오 스트림 얻은 후 프레임 단위로 저장
cap = utils.openVideoStream('https://www.youtube.com/watch?v=h4ILpWwU1LM')
# utils.extractFrames720p(cap, 120, 10, 30, './frames')
utils.extractFrames(cap, 120, 10, cap.get(cv2.CAP_PROP_FPS), './frames')
# 프레임 저장 경로
src_dir = './frames'
# 24씽 나눌 클립 폴더 경로
dst_root = './clips'

os.makedirs(dst_root, exist_ok=True)

frame_files = sorted([
    f for f in os.listdir(src_dir) if f.lower().endswith('.jpg')    
])

# 24 프레임으로 모델 학습 -> 24장씩 잘라서 저장
clip_len = 24
num_clips = len(frame_files) // clip_len

for i in range(num_clips):
    clip_dir = os.path.join(dst_root, f"clip_{i:03d}")
    os.makedirs(clip_dir, exist_ok=True)

    for j in range(clip_len):
        frame_idx = i * clip_len + j
        src_path = os.path.join(src_dir, frame_files[frame_idx])
        dst_path = os.path.join(clip_dir, f"{j:03d}.jpg")
        shutil.copy2(src_path, dst_path)

    print(f"clip_{i:03d} 저장 완료")

'https://www.youtube.com/watch?v=h4ILpWwU1LM'에서 30fps 비디오 스트림 URL을 가져오는 중...
🎥 선택된 해상도: 590p @ 30fps
URL: https://rr2---sn-n3cgv5qc5oq-bh2sy.googlevideo.com/videoplayback?expire=1752677804&ei=TGl3aNLZAcvX1d8Po5z16QI&ip=163.180.118.139&id=o-AAOdejJF_fsDo2FOGkGgcQJEHt43FMgQr456u1gUJ7uL&itag=136&aitags=133%2C134%2C135%2C136%2C160%2C242%2C243%2C244%2C247%2C278%2C298%2C299%2C302%2C303%2C308&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1752656204%2C&mh=J2&mm=31%2C26&mn=sn-n3cgv5qc5oq-bh2sy%2Csn-oguesnd6&ms=au%2Conr&mv=m&mvi=2&pl=19&rms=au%2Cau&gcr=kr&initcwndbps=5425000&bui=AY1jyLPJbY9YP5f2aFzRh271ovgSkFKdjwV_muqiokdaHRS3aE_b6bsIH62WqMghb4OyCCMBj3ZWDTyQ&vprv=1&svpuc=1&mime=video%2Fmp4&ns=BrzWowOqRKAtZfjNgC6G52cQ&rqh=1&gir=yes&clen=41029970&dur=299.566&lmt=1686381832025751&mt=1752655760&fvip=2&keepalive=yes&lmw=1&fexp=51544120&c=TVHTML5&sefc=1&txp=5432434&n=L6TFBHSMs9V4Vg&sparams=expire%2Cei%2Cip%2Cid%2Caitags%2Csource%2Crequiressl%2Cxpc%2Cgcr%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%

In [ ]:
import torch
from torch import nn

# 하이퍼파라미터
N_CLASSES = 4  # [1.0, 1.1, 1.2, 1.3]

# MAC이라면 mps
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 구조 정의 (학습 때와 동일)
model = torch.hub.load("facebookresearch/pytorchvideo", "x3d_s", pretrained=True)
model.blocks[-1].proj = nn.Linear(2048, N_CLASSES)
model = model.to(DEVICE)

# 학습된 가중치 불러오기
model.load_state_dict(torch.load("checkpoints/best_ep9.pth", map_location=DEVICE))  # best_epX.pth는 저장된 파일명
model.eval()

In [5]:
import os
import cv2
import torch
import torchvision.transforms as T
from pathlib import Path
import torch.nn.functional as F

# 1. 경로에서 프레임 이미지 리스트 얻기
def load_img_paths(frame_dir="./frames"):
    img_paths = sorted([
        str(Path(frame_dir) / f).replace("\\", "/")
        for f in os.listdir(frame_dir)
        if f.lower().endswith(".jpg")
    ])
    print(f"총 {len(img_paths)}개 프레임 로드 완료.")
    return img_paths

# 2. 프레임들을 클립 텐서로 변환 (24프레임 기준)
def preprocess_clip(img_paths, speed=1.0, num_frames=24, target_size=160):
    transform = T.Compose([
        T.ToPILImage(),
        T.Resize((target_size, target_size)),
        T.ToTensor()
    ])

    needed = int(num_frames * speed)
    frames = []
    for i in range(num_frames):
        idx = int(round(i * speed))
        if idx >= len(img_paths):
            print(f"프레임 인덱스 초과: idx={idx}, len={len(img_paths)}")
            return None
        img_path = img_paths[idx]
        img = cv2.imread(img_path)
        if img is None:
            print(f"이미지 로딩 실패: {img_path}")
            return None
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frames.append(transform(img))

    if len(frames) < num_frames:
        print("프레임 수 부족")
        return None

    clip = torch.stack(frames).permute(1, 0, 2, 3)  # (C, T, H, W)
    return clip.unsqueeze(0)  # (1, C, T, H, W)

# 3. 학습된 모델 로드
def load_trained_model(ckpt_path="checkpoints/best_ep10.pth", num_classes=4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("디바이스:", device)

    model = torch.hub.load("facebookresearch/pytorchvideo", "x3d_s", pretrained=True)
    model.blocks[-1].proj = torch.nn.Linear(2048, num_classes)
    model.load_state_dict(torch.load(ckpt_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model, device

# 4. 실제 예측 수행
def predict_speed(model, device, img_paths):
    clip_tensor = preprocess_clip(img_paths)
    if clip_tensor is None:
        print("❌ 클립 준비 실패")
        return

    clip_tensor = clip_tensor.to(device)
    with torch.no_grad():
        logits = model(clip_tensor)
        probs = F.softmax(logits, dim=1).squeeze().cpu().numpy()

    label_to_speed = {0: 1.0, 1: 1.1, 2: 1.2, 3: 1.3}
    for idx, prob in enumerate(probs):
        print(f"→ {label_to_speed[idx]:.1f}x : {prob*100:.2f}%")

    pred_idx = probs.argmax()
    pred_speed = label_to_speed[pred_idx]
    print(f"🎯 최종 예측된 배속: {pred_speed:.1f}x\n")

In [10]:
img_paths = load_img_paths("./clips/clip_009")  # frames 폴더에 프레임 이미지 존재해야 함

    # (2) 모델 로드
model, device = load_trained_model("./best_ep9.pth")  # 저장된 가중치 파일 경로

    # (3) 예측
predict_speed(model, device, img_paths)

총 24개 프레임 로드 완료.
디바이스: cuda


Using cache found in C:\Users\KHU/.cache\torch\hub\facebookresearch_pytorchvideo_main


→ 1.0x : 46.19%
→ 1.1x : 1.69%
→ 1.2x : 52.04%
→ 1.3x : 0.09%
🎯 최종 예측된 배속: 1.2x

